<span style="color:DarkBlue;font-size:32px">Preisvorhersage von Gebrauchtwagen </span>

<span style="color:DarkBlue;font-size:18px">Semesterabschließende schriftliche Ausarbeitung im Modul Machine Learning (SS2023)</span>

<span style="color:DarkBlue;font-size:18px"> Bearbeitet von M.Sc. Onur Yilmaz</span>

### **Inhaltsverzeichnis**

[Einleitung](#Einleitung)

[1. Datenaufbereitung](#Datenaufbereitung)

[1.1. Daten extrahieren](#Daten-extrahieren)

[1.2. Daten bereinigen](#Daten-bereinigen)

[1.3. Daten transformieren](#Daten-transformieren)

[2. Explorative Datenanalyse](#Explorative-Datenanalyse)

[2.1. Univariate Analyse](#Univariate-Analyse)

[2.2. Bivariate Analyse](#Bivariate-Analyse)

[2.3. Multivariate Analyse](#Multivariate-Analyse)

[2.4. Identifikation von Ausreißern](#Identifikation-von-Ausreißern)

[3. Feature Engineering](#Feature-Engineering)

[4. Modell Auswahl und Training](#Modell-Auswahl-und-Training)

[4.1. XGBoost](#XGBoost)

[5. Modell Bewertung](#Modell-Bewertung)

[6. Streamlit](#Streamlit)

[Literaturverzeichnis](#Literaturverzeichnis)


### <a id="Einleitung"></a>**Einleitung**

In der vorliegenden semesterabschließenden Arbeit wird ein aufbereiteter Datensatz von 100.000 gebrauchten Autoanzeigen aus dem Vereinigten Königreich analysiert. Der Datensatz wurde von der Plattform Kaggle bezogen - **https://www.kaggle.com/datasets/adityadesai13/used-car-dataset-ford-and-mercedes**.

Das Hauptziel dieser Arbeit ist die Wahl und Implementierung eines geeigneten maschinellen Lernmodells zur Preisvorhersage von Gebrauchtwagen.

Die Arbeit gliedert sich in verschiedene Abschnitte, beginnend mit der Datenaufbereitung, gefolgt von der explorativen Datenanalyse und dem Feature Engineering. Anschließend wird das passende Modell ausgewählt und trainiert, bevor es bewertet wird. Zusätzlich wird die Implementierung in Streamlit durchgeführt, um eine interaktive Webanwendung zur Darstellung der Ergebnisse zu erstellen.

Der Datensatz ist in einzelne Dateien nach Autoherstellern unterteilt und enthält Informationen wie Preis, Getriebe, Kilometerstand, Kraftstoffart, Kfz-Steuer, Verbrauch in Meilen pro Gallone (mpg) und Motorgröße.

**Das gesamte Projekt kann aus dem folgenden Repository geklont werden: https://www.github.com/ONURYI**

### <a id="Datenaufbereitung"></a>**1. Datenaufbereitung**

#### <a id="Daten-extrahieren"></a>**1.1. Daten extrahieren**

Die Datenaufbereitung stellt im Prozess des maschinellen Lernens oft einen entscheidenden, aber zugleich zeitaufwendigen Schritt dar. Sie umfasst diverse Techniken zur Säuberung, Transformation und Organisation der Rohdaten, um sie in ein für die Analyse passendes Format zu überführen. Diese Etappe legt das solide Fundament für die Erstellung eines Modells, da sie die Qualität und Relevanz der Daten gewährleistet. Dies trägt maßgeblich zur Präzision und Leistungsfähigkeit des endgültigen Modells bei [2].

Im Kontext unseres Projektes verwenden wir folgende CSV-Dateien (CSV = Comma-Separated Values):

- audi.csv
- bmw.csv
- ford.csv
- hyundai.csv
- mercedes.csv
- skoda.csv
- toyota.csv
- vauxhall.csv (unbenannt in opel)
- vw.csv

Diese Dateien sind im Ordner **/data** gespeichert und werden nach dem Aufbereitungs- und Bereinigungsprozess im Ordner **/data (clean)** als einzige zusammengefasste CSV-Datei abgelegt.

Für diesen Abschnitt wird ausschließlich auf die weit verbreitete **Pandas-Bibliothek** zurückgegriffen, welche sich als leistungsfähiges Werkzeug zur Datenmanipulation und -analyse etabliert hat. Hierbei lesen wir die Daten aus und stellen die Daten als sog. **DataFrames** dar, da sie eine standardisierte, zweidimensionale Struktur bieten, die das Speichern, Manipulieren und Analysieren von tabellarischen Daten vereinfacht, wodurch die Datenanalyse und -verarbeitung insgesamt effizienter und übersichtlicher wird [3].

In [ ]:
import pandas as pd

Wir lesen nun die ersten CSV-Files aus, uns erst einmal einen Überblick zu verschaffen:

In [ ]:
df_audi =  pd.read_csv('../data/audi.csv')
df_ford  =  pd.read_csv('../data/ford.csv')

In [ ]:
df_audi.head()

In [ ]:
df_ford.head()

Der Befehl *df.shape* gibt uns Auskunft über die zugehörige Dimension unseres DataFrames. Die erste Zahl, gibt die Anzahl der Zeilen und die zweite Zahl, die Anzahl der Spalten (auch Variable oder Merkmale genannt) wieder.

In [ ]:
print(df_audi.shape)
print(df_ford.shape)

Um schließlich einen umfassenden Datensatz zu erhalten, fassen wir diesen in einem DataFrame zusammen, wobei wir eine zusätzliche Spalte hinzugefügt haben, die als brand bezeichnet wird und die Marke des Modells enthält.

In [ ]:
file_paths = [
    '../data/audi.csv',
    '../data/bmw.csv',
    '../data/ford.csv',
    '../data/hyundai.csv',
    '../data/mercedes.csv',
    '../data/skoda.csv',
    '../data/toyota.csv',
    '../data/opel.csv',
    '../data/vw.csv',
]

all_dataframes = []

# Durchlaufe alle Dateipfade und lade die CSV-Dateien
for file_path in file_paths:
    df = pd.read_csv(file_path)
    brand = file_path.split('/')[-1].split('.')[0]  # Extrahiere die Marke aus dem Dateinamen
    df['brand'] = brand  # Füge eine Spalte mit der Marke hinzu
    # Ändere die Reihenfolge der Spalten, um "brand" an den Anfang zu setzen
    df = df[['brand'] + [col for col in df.columns if col != 'brand']]
    all_dataframes.append(df)

df = pd.concat(all_dataframes, ignore_index=True)

In [ ]:
df.head()

In [ ]:
df.shape

#### <a id="Daten-bereinigen"></a>**1.2. Daten bereinigen**

Um redundate Daten vorzubeugen müssen wir vorab unsere Daten auf Duplikate prüfen:

In [ ]:
duplicate_count = df.duplicated().sum()
print(duplicate_count)

df = df.drop_duplicates()

df.shape

Insgesamt hat sich der Datensatz somit um 1457 Zeilen reduziert. 

Als nächsten Schritt prüfen wir den Datensatz anschließend auf fehlende Werte:

In [ ]:
missing_values = df.isnull().sum()
missing_values[missing_values > 0]

Es gibt fehlende Werte in den Spalten tax, mpg, und tax(£). 

Gerade die Spalte tax(£) scheint in einem großen Teil des Datensatzes zu fehlen, weshalb wir die Spalte entfernen können, aufgrund fehlendem Informationsgehalt.


Die anderen Spalten könnten wir die fehlenden Werte mit geeigneten Methoden behandeln, wie z.B. durch das Ausfüllen mit dem Durchschnittswert oder Median.

In [ ]:
df = df.drop(columns=['tax(£)'])
df['tax'].fillna(df['tax'].median(), inplace=True)
df['mpg'].fillna(df['mpg'].median(), inplace=True)

In [ ]:
df.isnull().sum()

Nun nachdem wir geprüft haben, dass auch die Daten keine fehlenden Werte (**missing values**) mehr enthalten, müssen wir nun schauen das unsere Daten insgesamt plausibel sind.

In [ ]:
print(df['year'].min())
print(df['year'].max())

Diese problematischen Zeilen müssen wir nun einmal filtern.

In [ ]:
df = df[(df['year'] != 1970) & (df['year'] != 2060)]

In [ ]:
df.shape

In [ ]:
unique_values_counts = df['transmission'].value_counts()

print("Einzelne eindeutige Werte und ihre Häufigkeit:")
print(unique_values_counts)

In [ ]:
unique_values_counts = df['fuelType'].value_counts()

print("Einzelne eindeutige Werte und ihre Häufigkeit:")
print(unique_values_counts)

#### <a id="Daten-transformieren"></a>**1.3. Daten transformieren**

Da es sich bei dem vorliegenden Datensatz um einen UK-Datensatz handelt, waren die Spalten und Zeilen ursprünglich für den englischsprachigen Raum konzipiert. Um jedoch eine bessere Interpretierbarkeit unserer Daten im deutschsprachigen Kontext zu gewährleisten, haben wir die Daten entsprechend transformiert. 

**Es ist wichtig zu betonen, dass die Konvertierung von Einheiten und die Abweichung von den Rohdaten problematisch sein können!**

Hierbei wurde [4] hinzugezogen um noch einmal auf die Richtigkeit der Funktionen zu überprüfen und nachzujustieren.

In [ ]:
# Funktion Umrechnung von (mpg) zu (l/100km)
def mpg_to_l_per_100km(mpg):
    return 235.215 / mpg

# Funktion Umrechnung Meilen zu km
def miles_to_km(miles):
    return miles * 1.60934

In [ ]:
df.columns = [
    'Marke', 'Modell', 'Baujahr', 'Preis (£)', 'Getriebe', 'Kilometerstand', 'Kraftstoffart', 'Steuer (£)', 'Verbrauch (l/100km)', 'Motorgröße (l)']

df['Getriebe'] = df['Getriebe'].map({'Manual': 'Manuell', 
                                     'Automatic': 'Automatik', 
                                     'Semi-Auto': 'Halbautomatisch',
                                     'Other': 'Andere'})

df['Kraftstoffart'] = df['Kraftstoffart'].map({'Petrol': 'Benzin', 
                                               'Diesel': 'Diesel', 
                                               'Hybrid': 'Hybrid',
                                               'Other': 'Andere',
                                               'Electric': 'Elektrisch'})

df['Kilometerstand'] = df['Kilometerstand'].apply(miles_to_km).astype(int)

df['Verbrauch (l/100km)'] = df['Verbrauch (l/100km)'].apply(mpg_to_l_per_100km).round(2)

df['Marke'] = df['Marke'].str.capitalize()

In [ ]:
# Wechselkurs von Pfund zu Euro festlegen
exchange_rate = 1.15

# Umrechnung der Preise und Steuern von Pfund in Euro
df['Preis (€)'] = df['Preis (£)'] * exchange_rate
df['Steuer (€)'] = df['Steuer (£)'] * exchange_rate

df['Preis (€)']


# Entfernen der alten Spalten in Pfund
df.drop(columns=['Preis (£)', 'Steuer (£)'], inplace=True)

In [ ]:
string_columns = ['Marke', 'Modell', 'Getriebe', 'Kraftstoffart']

# Leerzeichen in den ausgewählten Spalten entfernen
df[string_columns] = df[string_columns].apply(lambda x: x.str.replace(" ", ""))

In [ ]:
# Entfernen der unerwünschten Ausprägungen in 'Getriebe' und 'Kraftstoffart'
df = df[(df['Getriebe'] != 'Andere') & 
                 (df['Kraftstoffart'] != 'Elektrisch') & 
                 (df['Kraftstoffart'] != 'Andere')]


In [ ]:
df

Speichern unseres bereinigten Datensatzes in der Ordner **\data (clean)**:

In [ ]:
df.to_csv('../data (clean)/df_clean.csv', index=False)

### <a id="Explorative-Datenanalyse"></a>**2. Explorative Datenanalyse**


Der folgende Abschnitt stützt sich auf Konzepte und Methoden aus [5].

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df.dtypes

In [ ]:
df.describe().T

#### <a id="Daten-bereinigen"></a>**2.1. Univariate Analyse**

In der univariaten Analyse werden wir die Verteilung jedes Merkmals einzeln betrachten um uns einen Überblick über die Daten zu verschaffen. 

Hierfür werden wir sowohl numerische als auch kategoriale Merkmale analysieren.

In [ ]:
plt.figure(figsize=(10, 10))

numerical_columns = ['Baujahr', 'Kilometerstand', 'Verbrauch (l/100km)', 'Motorgröße (l)', 'Preis (€)', 'Steuer (€)']

# Erstellen von Subplots für jede numerische Variable
for i, col in enumerate(numerical_columns, 1):
    plt.subplot(2, 3, i)
    sns.histplot(df[col], bins=20, kde=True)
    plt.title(f'Verteilung von {col}')
    plt.xlabel(col)
    plt.ylabel('Anzahl')

plt.tight_layout()
plt.show()

Die Histogramme zeigen die Verteilung der einzelnen numerischen Merkmale:

- Baujahr: Die meisten Autos wurden zwischen 2016 und 2020 gebaut.

- Kilometerstand: Die Verteilung ist rechtsschief, wobei die meisten Autos einen niedrigen Kilometerstand haben.

- Verbrauch (l/100km): Die meisten Autos haben einen Verbrauch zwischen 3 und 6 Litern pro 100 Kilometer. 

- Motorgröße (l): Die meisten Autos haben eine Motorgröße zwischen 1 und 2 Litern.

- Preis (€): Die Verteilung des Preises ist rechtsschief, wobei die meisten Autos im niedrigeren Preissegment liegen.

- Steuer (€): Die Steuern variieren, wobei die meisten Autos Steuern im Bereich von 100 bis 200 € haben.

Nun betrachten wir noch die Verteilungen der kategorischen Merkmale: "Marke", "Getriebe" und "Kraftstoffart". 

In [ ]:
plt.figure(figsize=(10, 10))

categorical_columns = ['Marke', 'Getriebe', 'Kraftstoffart']

# Erstellen von Subplots für jede kategorische Variable
for i, col in enumerate(categorical_columns, 1):
    plt.subplot(2, 2, i)
    sns.countplot(data=df, y=col, order=df[col].value_counts().index)
    plt.title(f'Verteilung von {col}')
    plt.xlabel('Anzahl')
    plt.ylabel(col)

plt.tight_layout()
plt.show()


Da die Variable Modell zu viele Ausprägungen besitzt beschränken wir uns auf die Top-3-Modelle für jede der Top-5-Marken in den Daten. 

In [ ]:
top_5_brands = df['Marke'].value_counts().nlargest(5).index.tolist()

plt.figure(figsize=(8, 8))
for i, brand in enumerate(top_5_brands, 1):
    top_3_models = df[df['Marke'] == brand]['Modell'].value_counts().nlargest(3)
    plt.subplot(2, 3, i)
    sns.barplot(x=top_3_models.index, y=top_3_models.values, palette='viridis')
    plt.title(f'Top-3-Modelle für {brand}')
    plt.xlabel('Modell')
    plt.ylabel('Anzahl')

plt.tight_layout()
plt.show()

#### <a id="Daten-bereinigen"></a>**2.2. Bivariate Analyse**

Die Korrelationsanalyse ist eine spezifische Form der bivariaten Analyse, bei der wie die lineare Beziehung zwischen zwei Merkmalen mit Hilfe des sog. *Pearson-Korrelationskoeffizient* messen.

In [ ]:
# Erstellen von Scatterplots für Preis gegen andere numerische Merkmale
numerical_columns.remove('Preis (€)')

plt.figure(figsize=(15, 10))
for i, feature in enumerate(numerical_columns, 1):
    plt.subplot(2, 3, i)
    sns.scatterplot(x=df[feature], y=df['Preis (€)'], alpha=0.5, edgecolor=None)
    plt.title(f'Preis vs. {feature}')
    plt.xlabel(feature)
    plt.ylabel('Preis (€)')

plt.tight_layout()
plt.show()


- Preis vs. Baujahr: Es scheint einen positiven Zusammenhang zwischen dem Baujahr und dem Preis zu geben. Neuere Autos tendieren dazu, teurer zu sein.

- Preis vs. Kilometerstand: Es gibt einen negativen Zusammenhang zwischen Kilometerstand und Preis. Autos mit höherem Kilometerstand sind tendenziell günstiger.

In [ ]:
categorical_pairings = [('Preis (€)', 'Getriebe'), ('Preis (€)', 'Kraftstoffart')]

plt.figure(figsize=(10, 5))
for i, (y, x) in enumerate(categorical_pairings, 1):
    plt.subplot(1, 2, i)
    sns.boxplot(x=df[x], y=df[y], palette='viridis')
    plt.title(f'{y} nach {x}')
    plt.xlabel(x)
    plt.ylabel(y)

plt.tight_layout()
plt.show()


- Preis vs. Getriebe: Autos mit Automatikgetriebe scheinen im Durchschnitt teurer zu sein als solche mit manuellem Getriebe.

- Preis vs. Kraftstoffart: Die Preise scheinen je nach Kraftstoffart unterschiedlich zu sein. Diesel-Autos sind tendenziell etwas teurer als Benziner.

Die multivariate Analyse geht über die bivariate Analyse hinaus und untersucht die Beziehungen zwischen mehr als zwei Merkmalen gleichzeitig. 

In [ ]:
correlation_matrix = df.corr(numeric_only=True)

plt.figure(figsize=(7, 7))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Korrelationsmatrix')
plt.show()

# Zeigen der Korrelationen mit dem Preis
correlation_with_price = correlation_matrix['Preis (€)'].sort_values(ascending=False)
correlation_with_price


In [ ]:
from scipy.stats import pearsonr

numerical_features = ['Baujahr', 'Kilometerstand', 'Verbrauch (l/100km)', 'Motorgröße (l)', 'Preis (€)', 'Steuer (€)']

# Berechnen der Korrelationskoeffizienten und p-Werte
correlation_p_values = {feature: pearsonr(df['Preis (€)'], df[feature]) for feature in numerical_features if feature != 'Preis (€)'}

# Umwandeln in ein DataFrame für eine bessere Darstellung
correlation_p_values_df = pd.DataFrame.from_dict(correlation_p_values, orient='index', columns=['Korrelation', 'p-Wert'])
correlation_p_values_df


Alle p-Werte sind 0, was bedeutet, dass die Korrelationen statistisch signifikant sind.

#### <a id="Daten-bereinigen"></a>**2.4. Multivariate Analyse**

In [ ]:
plt.figure(figsize=(10, 8))
scatter = sns.scatterplot(data=df, x='Kilometerstand', y='Preis (€)', hue='Getriebe', 
                          palette={'Manuell':'green', 'Automatik':'blue', 'Halbautomatisch':'red'}, alpha=0.7, edgecolor=None)
plt.title('Beziehung zwischen Kilometerstand, Preis und Getriebe')
plt.xlabel('Kilometerstand')
plt.ylabel('Preis (€)')

legend_labels = ['Manuell', 'Automatik', 'Halbautomatisch']
legend_colors = ['green', 'blue', 'red']
markers = [plt.Line2D([0,0],[0,0],color=color, marker='o', linestyle='') for color in legend_colors]
plt.legend(markers, legend_labels, title='Getriebe', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.show()

Jeder Punkt auf dem Diagramm repräsentiert ein Auto. Die horizontale Position des Punktes zeigt den Kilometerstand des Autos, während die vertikale Position den Preis des Autos zeigt.

Die Farbe des Punktes zeigt das Getriebe des Autos. Laut der Legende auf der rechten Seite repräsentieren grüne Punkte Autos mit manuellem Getriebe, blaue Punkte repräsentieren Autos mit Automatikgetriebe und rote Punkte repräsentieren Autos mit halbautomatischem Getriebe.

### <a id="Datenaufbereitung"></a>**3. Feauture Engineering**

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Entfernen der Spalte "Steuer (€)"
df.drop(columns='Steuer (€)', inplace=True)

# Anwenden des Label Encodings auf die kategorischen Spalten
label_columns = ['Marke', 'Modell', 'Getriebe', 'Kraftstoffart']
encoders = {}
for column in label_columns:
    encoder = LabelEncoder()
    df[column] = encoder.fit_transform(df[column])
    encoders[column] = encoder

# Berechnen des Alters der Autos
current_year = 2023
df['Alter'] = current_year - df['Baujahr']
df.drop(columns='Baujahr', inplace=True)  # Baujahr entfernen

# Trennen von Features und Zielvariable
X_feature_engineered = df.drop(columns=['Preis (€)'])
y_target_variable = df['Preis (€)']

# Anzeigen der ersten paar Zeilen der transformierten Daten
X_feature_engineered.head(), y_target_variable.head()

### <a id="Datenaufbereitung"></a>**4. Modell Training und Auswahl**

### **Literaturverzeichnis**

[1] https://www.kaggle.com/datasets/adityadesai13/used-car-dataset-ford-and-mercedes (01.08.2023)

[2] Müller, A. C., & Guido, S. (2021). Introduction to Machine Learning with Python: A Guide for Data Scientists (2nd ed.). O'Reilly Media

[3] McKinney, W. (2017). Python for Data Analysis: Data Wrangling with Pandas, NumPy, and IPython (2nd ed.). O'Reilly Media.

[4] OpenAI. (2023). Persönliche Kommunikation mit OpenAI's GPT-3.5 Modell. (01.08.2023)
 
[5] Peter Bruce, Andrew Bruce, Peter Gedeck. Praktische Statistik für Data Scientists: 50+ essenzielle Konzepte mit R und Python, 2021.